In [518]:
import numpy as np
import pandas as pd
from collections import Counter
from scipy import stats
import seaborn as sns; sns.set()
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from collections import defaultdict

In [152]:
def filter_df(df, species, epitopes, chains, cdr3_len):
    return df[(df['species'].isin(species)) &
              (df['antigen.epitope'].isin(epitopes)) &
              (df['gene'].isin(chains)) &
              (df['cdr3_len'] == cdr3_len)].copy()

In [10]:
def get_concervative_pos(df, q):
    start_pos = np.percentile(df['v.end'], q=q, interpolation='nearest')
    end_pos = np.percentile(df['j.start'], q=(100-q), interpolation='nearest') - 1
    return start_pos, end_pos

In [208]:
AMINO_ACIDS = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
AA_2_POS = {'A': 0, 'R': 1, 'N': 2, 'D': 3, 'C': 4,
            'Q': 5, 'E': 6, 'G': 7, 'H': 8, 'I': 9,
            'L': 10, 'K': 11, 'M': 12, 'F': 13, 'P': 14,
            'S': 15, 'T': 16, 'W': 17, 'Y': 18, 'V': 19}
AA_N = 20

def get_blosum_matrix(df, start_pos, end_pos):
    cdr3_full = df.cdr3.apply(lambda x: pd.Series(list(x))).values
    cdr3 = cdr3_full[:, start_pos:(end_pos+1)]

    # Frequency table
    F_ij = np.zeros([AA_N, AA_N])
    for j in range(0, cdr3.shape[1]):
        column_acids = cdr3[:, j]
        cnt = Counter(column_acids)
        for i_acid, i in AA_2_POS.items():
            for j_acid, j in AA_2_POS.items():
                if (i_acid == j_acid) and cnt[i_acid] > 1:
                    F_ij[i, j] += ((cnt[i_acid] * (cnt[i_acid] - 1)) / 2)
                else:
                    F_ij[i, j] += cnt[i_acid] * cnt[j_acid]
    
    # Observed probability
    lower_Tr = np.tril(np.ones([AA_N, AA_N]))
    np.fill_diagonal(lower_Tr, 1)
    total_pairs = np.sum(lower_Tr * F_ij)
    Q_ij = F_ij / total_pairs

    # Expected probability
    P_i = np.zeros(AA_N)
    for i_acid, i in AA_2_POS.items():
        P_i[i] = Q_ij[i, i] + (np.sum(Q_ij[i, :]) - Q_ij[i, i])/2
    E_ij = np.zeros([AA_N, AA_N])
    for i in np.arange(0, AA_N):
        for j in np.arange(0, AA_N):
            if i == j:
                E_ij[i, j] = P_i[i] * P_i[i]
            else:
                E_ij[i, j] = 2 * P_i[i] * P_i[j]
    
    # The log-odds ration
    L = np.zeros([AA_N, AA_N])
    for i in np.arange(0, AA_N):
        for j in np.arange(0, AA_N):
            if (Q_ij[i, j] == 0) or (E_ij[i, j] == 0):
                L[i, j] = 1
            else:
                L[i, j] = Q_ij[i, j] / E_ij[i, j]
    L = np.round(np.log2(L) * 2)
    return L

In [491]:
def spearman_corr(M1, M2):
    return stats.spearmanr(M1.values.flatten(), M2.values.flatten())
def pearson_corr(M1, M2):
    return stats.pearsonr(M1.values.flatten(), M2.values.flatten())

## Preparing stage

In [313]:
blosum62 = pd.read_csv('blosum62.txt', sep=r'\s+', index_col=0)
blosum62 = blosum62.loc[AMINO_ACIDS, AMINO_ACIDS]

#### Database preparing

In [202]:
vdjdb_slim_df = pd.read_csv('../vdjdb-dump/vdjdb.slim.txt', sep='\t')

# Filter referencies from 10xgenomics.com
vdjdb_slim_df = vdjdb_slim_df[~vdjdb_slim_df['reference.id'].str.startswith('https://www.10xgenomics.com')]

# Filter bad positions of j.start and v.end
vdjdb_slim_df = vdjdb_slim_df[(vdjdb_slim_df['v.end'] < vdjdb_slim_df['j.start']) & (vdjdb_slim_df['v.end'] > 0)]

# add CDR3 len
vdjdb_slim_df['cdr3_len'] = vdjdb_slim_df.cdr3.str.len()

In [203]:
# Percentiles of samples number per species and gene. 
ept_df = vdjdb_slim_df.groupby(['species', 'gene', 'antigen.epitope', 'cdr3_len'])['cdr3'].count().reset_index()
ept_df.columns = ['species', 'gene', 'antigen.epitope', 'cdr3_len', 'samples_cnt']

In [204]:
# Leave just epitopes with >= 30 CDR3 samples
good_epitopes = ept_df[ept_df.samples_cnt >= 30][['species', 'gene', 'antigen.epitope', 'cdr3_len', 'samples_cnt']]
vdjdb_slim_df = vdjdb_slim_df.merge(good_epitopes, how='inner', suffixes=['', '_r'],
                                    left_on=['species', 'gene', 'antigen.epitope', 'cdr3_len'],
                                    right_on=['species', 'gene', 'antigen.epitope', 'cdr3_len'])


In [536]:
IMGT_classes = [
    ('acidic', ['D', 'E']),
    ('aliphatic', ['A', 'I', 'L', 'V']),
    ('amide', ['N', 'Q']),
    ('aromatic', ['F', 'W', 'Y']),
    ('basic', ['H', 'K', 'R']),
    ('G', ['G']),
    ('hydroxyl', ['S', 'T']),
    ('P', ['P']),
    ('sulfur', ['C', 'M'])
]

IMGT_colors =  [
    '#1f77b4', '#ff7f0e', '#2ca02c',
    '#d62728', '#9467bd', '#8c564b',
    '#e377c2', '#7f7f7f', '#bcbd22'
]

def get_pca_imgt_traces(matrice, showlegend=True):
    pca = PCA(n_components=2)
    pca.fit(matrice.values)
    traces = []
    for num, (imgt_class, aa_v) in enumerate(IMGT_classes):
        components = pca.transform(matrice.loc[aa_v, :])
        traces.append(go.Scatter(x=components[:,0], y=components[:,1],
                                 mode='markers+text', text=aa_v,
                                 marker={'size': 17, 'opacity':0.5},
                                 line={'color': IMGT_colors[num]},
                                 name=imgt_class,
                                 showlegend=showlegend))
    return traces

def plot_pca_IMGT(matrice2params, rows, cols, titles):
    fig = make_subplots(rows=rows, cols=cols, subplot_titles=titles)
    for matrice, row, col in matrice2params:
        if row == 1 and col == 1:
            traces = get_pca_imgt_traces(matrice)
        else:
            traces = get_pca_imgt_traces(matrice, showlegend=False)
        for t in traces:
            fig.append_trace(t, row=row, col=col)
    for row in range(rows):
        for col in range(cols):
            fig.update_xaxes(title_text="pca_x", row=row+1, col=col+1)
            fig.update_yaxes(title_text="pca_y", row=row+1, col=col+1)
    fig.update_layout(plot_bgcolor='rgb(248,248,248)')
    fig.show()
    
def get_mds_imgt_traces(matrice, showlegend=True):
    mds = MDS(n_components=2)
    components = mds.fit_transform(matrice.values)
    class2x = defaultdict(list)
    class2y = defaultdict(list)
    for imgt_class, aa_v in IMGT_classes:
        for aa in aa_v:
            x, y = components[AA_2_POS[aa]]
            class2x[imgt_class].append(x)
            class2y[imgt_class].append(y)
    traces = []
    for num, (imgt_class, aa_v) in enumerate(IMGT_classes):
        traces.append(go.Scatter(x=class2x[imgt_class], y=class2y[imgt_class],
                                 mode='markers+text', text=aa_v,
                                 marker={'size': 17, 'opacity':0.5},
                                 line={'color': IMGT_colors[num]},
                                 name=imgt_class,
                                 showlegend=showlegend))
    return traces

def plot_mds_IMGT(matrice2params, rows, cols, titles):
    fig = make_subplots(rows=rows, cols=cols, subplot_titles=titles)
    for matrice, row, col in matrice2params:
        if row == 1 and col == 1:
            traces = get_mds_imgt_traces(matrice)
        else:
            traces = get_mds_imgt_traces(matrice, showlegend=False)
        for t in traces:
            fig.append_trace(t, row=row, col=col)
    for row in range(rows):
        for col in range(cols):
            fig.update_xaxes(title_text="mds_x", row=row+1, col=col+1)
            fig.update_yaxes(title_text="mds_y", row=row+1, col=col+1)
    fig.update_layout(plot_bgcolor='rgb(248,248,248)')
    fig.show()

#### Build matrices for all good epitopes 

In [205]:
hs_tra_samples = good_epitopes[(good_epitopes.species == 'HomoSapiens') & (good_epitopes.gene == 'TRA')].shape[0]
hs_trb_samples = good_epitopes[(good_epitopes.species == 'HomoSapiens') & (good_epitopes.gene == 'TRB')].shape[0]
mm_tra_samples = good_epitopes[(good_epitopes.species == 'MusMusculus') & (good_epitopes.gene == 'TRA')].shape[0]
mm_trb_samples = good_epitopes[(good_epitopes.species == 'MusMusculus') & (good_epitopes.gene == 'TRB')].shape[0]
print(f"HomoSapiens good epitopes (TRA): {hs_tra_samples}")
print(f"HomoSapiens good epitopes (TRB): {hs_trb_samples}")
print(f"MusMusculus good epitopes (TRA): {mm_tra_samples}")
print(f"MusMusculus good epitopes (TRB): {mm_trb_samples}")

HomoSapiens good epitopes (TRA): 27
HomoSapiens good epitopes (TRB): 77
MusMusculus good epitopes (TRA): 15
MusMusculus good epitopes (TRB): 18


In [511]:
matrices = {}

for r in good_epitopes.to_records():
    species, chain, epitope, cdr3_len = r[1], r[2], r[3], r[4]
    df = filter_df(vdjdb_slim_df, [species], [epitope], [chain], cdr3_len)
    start_pos, end_pos = get_concervative_pos(df, 80)
    L = get_blosum_matrix(df, cdr3_start_pos, cdr3_end_pos)
    result = pd.DataFrame(L, index=AMINO_ACIDS, columns=AMINO_ACIDS)
    matrices[(species, chain, epitope, cdr3_len)] = result
    print(f"{species}, {chain}, {epitope}, {cdr3_len} processed!")

HomoSapiens, TRA, FRDYVDRFYKTLRAEQASQE, 12 processed!
HomoSapiens, TRA, GILGFVFTL, 10 processed!
HomoSapiens, TRA, GILGFVFTL, 11 processed!
HomoSapiens, TRA, GILGFVFTL, 12 processed!
HomoSapiens, TRA, GILGFVFTL, 13 processed!
HomoSapiens, TRA, GILGFVFTL, 14 processed!
HomoSapiens, TRA, GILGFVFTL, 15 processed!
HomoSapiens, TRA, GILGFVFTL, 16 processed!
HomoSapiens, TRA, GILGFVFTL, 17 processed!
HomoSapiens, TRA, GILGFVFTL, 18 processed!
HomoSapiens, TRA, GILGFVFTL, 19 processed!
HomoSapiens, TRA, GLIYNRMGAVTTEV, 14 processed!
HomoSapiens, TRA, LLLGIGILV, 11 processed!
HomoSapiens, TRA, LLLGIGILV, 12 processed!
HomoSapiens, TRA, LLLGIGILV, 13 processed!
HomoSapiens, TRA, LLLGIGILV, 14 processed!
HomoSapiens, TRA, LLLGIGILV, 15 processed!
HomoSapiens, TRA, LLWNGPMAV, 10 processed!
HomoSapiens, TRA, NLVPMVATV, 10 processed!
HomoSapiens, TRA, NLVPMVATV, 11 processed!
HomoSapiens, TRA, NLVPMVATV, 12 processed!
HomoSapiens, TRA, NLVPMVATV, 13 processed!
HomoSapiens, TRA, NLVPMVATV, 14 proces

## Top 3 epitopes 

1) most common length

In [503]:
good_epitopes[(good_epitopes.species == 'HomoSapiens')
              & (good_epitopes.gene == 'TRB')].sort_values('samples_cnt', ascending=False)[0:3]

,species,gene,antigen.epitope,cdr3_len,samples_cnt
542,HomoSapiens,TRB,GILGFVFTL,13,1089
859,HomoSapiens,TRB,NLVPMVATV,15,1022
858,HomoSapiens,TRB,NLVPMVATV,14,903


In [505]:
M1 = matrices[('HomoSapiens', 'TRB', 'GILGFVFTL', 13)]
M2 = matrices[('HomoSapiens', 'TRB', 'NLVPMVATV', 15)]
M3 = matrices[('HomoSapiens', 'TRB', 'NLVPMVATV', 14)]

In [507]:
pearson_corr(M1, M2)

(0.0994823019148526, 0.046772519931885845)

In [509]:
pearson_corr(M2, M3)

(0.18810457493383512, 0.00015423402457194086)

In [534]:
plot_mds_IMGT([(blosum62, 1, 1), (M1, 1, 2)], rows=1, cols=2,
              titles=['BLOSUM62', 'HomoSapiens, TRB, GILGFVFTL, 14'])

/Users/nediolus/anaconda3/lib/python3.7/site-packages/sklearn/manifold/mds.py:421: UserWarning:

The MDS API has changed. ``fit`` now constructs an dissimilarity matrix from data. To use a custom dissimilarity matrix, set ``dissimilarity='precomputed'``.

/Users/nediolus/anaconda3/lib/python3.7/site-packages/sklearn/manifold/mds.py:421: UserWarning:

The MDS API has changed. ``fit`` now constructs an dissimilarity matrix from data. To use a custom dissimilarity matrix, set ``dissimilarity='precomputed'``.



## Compare HomoSapiens TRA/TRB matrices

In [498]:
tra_matrices = []
tra_matrices_samples_n = []
tra_epitopes = good_epitopes[(good_epitopes.species == 'HomoSapiens') & (good_epitopes.gene == 'TRA')].values
for species, chain, epitope, e_len, n_samples in tra_epitopes:
    tra_matrices.append(matrices[(species, chain, epitope, e_len)])
    tra_matrices_samples_n.append(n_samples)

trb_matrices = []
trb_matrices_samples_n = []
trb_epitopes = good_epitopes[(good_epitopes.species == 'HomoSapiens') & (good_epitopes.gene == 'TRB')].values
for species, chain, epitope, e_len, n_samples in trb_epitopes:
    trb_matrices.append(matrices[(species, chain, epitope, e_len)])
    trb_matrices_samples_n.append(n_samples)

Average matrices values

In [499]:
avg_tra_matrix = pd.DataFrame(index=AMINO_ACIDS, columns=AMINO_ACIDS).fillna(0)
for matrix in tra_matrices:
    avg_tra_matrix += matrix
avg_tra_matrix = (avg_tra_matrix / len(tra_matrices)).round()

avg_trb_matrix = pd.DataFrame(index=AMINO_ACIDS, columns=AMINO_ACIDS).fillna(0)
for matrix in trb_matrices:
    avg_trb_matrix += matrix
avg_trb_matrix = (avg_trb_matrix / len(trb_matrices)).round()

In [500]:
print("Spearman correlation")
print(f"TRA vs TRB: {spearman_corr(avg_tra_matrix, avg_trb_matrix)}")
print(f"TRA vs BLOSUM62: {spearman_corr(avg_tra_matrix, blosum62)}")
print(f"TRB vs BLOSUM62: {spearman_corr(avg_trb_matrix, blosum62)}")
print()
print("Pearson correlation")
print(f"TRA vs TRB: {pearson_corr(avg_tra_matrix, avg_trb_matrix)}")
print(f"TRA vs BLOSUM62: {pearson_corr(avg_tra_matrix, blosum62)}")
print(f"TRB vs BLOSUM62: {pearson_corr(avg_trb_matrix, blosum62)}")

Spearman correlation
TRA vs TRB: SpearmanrResult(correlation=0.3630255512952816, pvalue=6.62953549816995e-14)
TRA vs BLOSUM62: SpearmanrResult(correlation=0.24432353743606133, pvalue=7.562225818168715e-07)
TRB vs BLOSUM62: SpearmanrResult(correlation=0.221479715218314, pvalue=7.771525599360681e-06)

Pearson correlation
TRA vs TRB: (0.4601860769226917, 2.336927394892734e-22)
TRA vs BLOSUM62: (0.43572905891297864, 5.773886013234608e-20)
TRB vs BLOSUM62: (0.5018257708367269, 6.670613164762445e-27)


In [535]:
plot_mds_IMGT([(blosum62, 1, 1), (avg_tra_matrix, 1, 2), (avg_trb_matrix,1 , 3)],
              rows=1, cols=3,
              titles=['BLOSUM62', 'HomoSapiens, TRA', 'HomoSapiens, TRB'])

/Users/nediolus/anaconda3/lib/python3.7/site-packages/sklearn/manifold/mds.py:421: UserWarning:

The MDS API has changed. ``fit`` now constructs an dissimilarity matrix from data. To use a custom dissimilarity matrix, set ``dissimilarity='precomputed'``.

/Users/nediolus/anaconda3/lib/python3.7/site-packages/sklearn/manifold/mds.py:421: UserWarning:

The MDS API has changed. ``fit`` now constructs an dissimilarity matrix from data. To use a custom dissimilarity matrix, set ``dissimilarity='precomputed'``.

/Users/nediolus/anaconda3/lib/python3.7/site-packages/sklearn/manifold/mds.py:421: UserWarning:

The MDS API has changed. ``fit`` now constructs an dissimilarity matrix from data. To use a custom dissimilarity matrix, set ``dissimilarity='precomputed'``.



#### Simple example, Homo Sapiens, GIL antigen, TRB, Len14, GILGFVFTL

In [267]:
species, chain, epitope, cdr3_len = 'HomoSapiens', 'TRB', 'GILGFVFTL', 14
df = filter_df(vdjdb_slim_df, [species], [epitope], [chain], cdr3_len)
start_pos, end_pos = get_concervative_pos(df, 80)
L = get_blosum_matrix(df, cdr3_start_pos, cdr3_end_pos)
result = pd.DataFrame(L, index=AMINO_ACIDS, columns=AMINO_ACIDS)
result

,A,R,N,D,C,Q,E,G,H,I,L,K,M,F,P,S,T,W,Y,V
A,0.0,0.0,1.0,0.0,0.0,-0.0,0.0,-0.0,0.0,-1.0,-0.0,-0.0,-1.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0
R,0.0,1.0,-1.0,0.0,0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,-0.0,0.0,0.0,-0.0,0.0
N,1.0,-1.0,2.0,-0.0,0.0,-1.0,0.0,-1.0,1.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,2.0,0.0
D,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-1.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-1.0,-0.0
C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Q,-0.0,-0.0,-1.0,0.0,0.0,1.0,-0.0,-0.0,-0.0,1.0,1.0,0.0,1.0,0.0,1.0,-0.0,-0.0,0.0,-0.0,-0.0
E,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0
G,-0.0,0.0,-1.0,0.0,0.0,-0.0,0.0,1.0,-0.0,-2.0,-1.0,-0.0,-1.0,-1.0,-1.0,0.0,-0.0,-0.0,-1.0,-0.0
H,0.0,-0.0,1.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,1.0,-0.0
I,-1.0,-0.0,-2.0,-1.0,0.0,1.0,-0.0,-2.0,-0.0,2.0,2.0,1.0,2.0,1.0,2.0,-1.0,-1.0,1.0,0.0,0.0


In [268]:
print(f'N(D)N start position: {start_pos}, end position: {end_pos}')

N(D)N start position: 5, end position: 7


In [270]:
df['v_len_arr'] = df['v.end']
df['cdr3_len_arr'] = df['j.start'] - df['v.end']
df['j_len_arr'] = CDR3_LEN - df['j.start']

In [274]:
plot_sample_n = 100
plot_df = df.sample(plot_sample_n, random_state=42).sort_values(['cdr3_len_arr', 'v_len_arr'])
traces = []
traces.append(go.Bar(x=plot_df.v_len_arr, orientation='h', name='V'))
traces.append(go.Bar(x=plot_df.cdr3_len_arr, orientation='h', name='N(D)N'))
traces.append(go.Bar(x=plot_df.j_len_arr, orientation='h', name='J'))
traces.append(go.Scatter(x=[start_pos, start_pos], y=[0, plot_sample_n],
                         name='N(D)N start, 80%', line={'color': 'black'}))
traces.append(go.Scatter(x=[end_pos+1, end_pos+1], y=[0, plot_sample_n],
                         name='N(D)N end, 80%', line={'color': 'black'}))
fig = go.Figure(data=traces)
fig.update_layout(barmode='stack', title='HomoSapiens, GIL, TRB, Len: 14')
fig.show()